# Descarga de excels

In [ ]:
from datetime import datetime
Inicio_programa=datetime.now()

In [ ]:
import os
import tabula
import re
from openpyxl import Workbook,load_workbook
import pandas as pd
import shutil
from natsort import natsorted

In [ ]:
path_local="/home/rodrigo/Scrapper_Sueldos_Municipales"

# Verificación de carpetas

In [ ]:
path_dhea=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/pdf"
lista=natsorted(os.listdir(path_dhea))
for i in lista:
    if re.search("Sueldos",i) is None:
        lista.remove(i)

In [ ]:
lista

In [ ]:
pdf_a_convertir=lista[-1]

# Crea la carpeta donde se van a guardar los excel en crudo

In [ ]:
path_dhea=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/pdf/"+pdf_a_convertir
Lista_pdf_nuevos=natsorted(os.listdir(path_dhea))

Lista_util=[re.sub("\.pdf","",i) for i in Lista_pdf_nuevos]


try :
    os.mkdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza")
    for i in Lista_util:
        os.mkdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza"+"/"+i)
except :
    pass

for i in enumerate(Lista_pdf_nuevos):
    variable = True
    n=0
    df = tabula.read_pdf(path_dhea+"/"+i[1], pages = "all")
    while (variable == True):
        try :
            df[n].to_excel(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza"+"/"+Lista_util[i[0]]+"/"+Lista_util[i[0]]+"_imperfecto_"+str(n)+".xlsx")
            n+=1
        except :
            variable=False

In [ ]:
carpeta_especifica=re.sub("_Sueldos","",pdf_a_convertir)
carpeta_especifica

# Acomodo de excel sucios

In [ ]:
path_carpetas_excels_sucios=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza"
carpetas_excels_sucios=natsorted(os.listdir(path_carpetas_excels_sucios))

for i in enumerate(carpetas_excels_sucios):
    path_excels_imperfectos= path_carpetas_excels_sucios+"/"+ i[1]
    archivos_pdf=natsorted(os.listdir(path_excels_imperfectos))
    n=0
    for j in enumerate(archivos_pdf):
        workbook_malo = load_workbook(filename=path_excels_imperfectos+"/"+j[1])
        sheet_malo= workbook_malo.active
        sheet_malo.delete_cols(1)
        celda_1=sheet_malo.cell(row=1, column=1).value
        celda_2=sheet_malo.cell(row=1, column=2).value
        if re.search("[a-z]",celda_1) is not None :
            os.remove(path_excels_imperfectos+"/"+j[1])
            n+=1
        elif re.search("[0-9]",celda_1) is not None and re.search("Unnamed: 0",celda_2) is not None:
            inicio=re.search("[0-9]",celda_1).span()[0]
            nombre=celda_1[:inicio]
            numero=celda_1[inicio:]
            sheet_malo["A1"] = nombre
            sheet_malo["B1"] = numero
            sheet_malo.insert_rows(1)
            sheet_malo["A1"],sheet_malo["B1"],sheet_malo["C1"],sheet_malo["D1"],sheet_malo["E1"],sheet_malo["F1"]="apellido_y_nombre","sueldo_bruto","sueldo_neto","costo_laboral","planta","funcion"
            workbook_malo.save(filename=path_excels_imperfectos+"/"+j[1])
            os.rename(path_excels_imperfectos+"/"+j[1] , path_excels_imperfectos+"/"+i[1]+"_perfecto_"+str(n)+".xlsx")
            n+=1
        else :
            sheet_malo.insert_rows(1)
            sheet_malo["A1"],sheet_malo["B1"],sheet_malo["C1"],sheet_malo["D1"],sheet_malo["E1"],sheet_malo["F1"]="apellido_y_nombre","sueldo_bruto","sueldo_neto","costo_laboral","planta","funcion"
            workbook_malo.save(filename=path_excels_imperfectos+"/"+j[1])
            os.rename(path_excels_imperfectos+"/"+j[1],path_excels_imperfectos+"/"+i[1]+"_perfecto_"+str(n)+".xlsx")
            n+=1

# Crea la carpeta donde se almacenan los excel

In [ ]:
# Algo asi va la onda, Hay que resolver el problema de iterar los nombres en los excels
# Hay que hacer que se "sincronice" el numero de lista de nombres (i) con la de k,asi solo analizar un directorio por iteracion

path_viejo=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
try:
    os.mkdir(path_viejo)
except:
    pass
for i in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/secciones")):
    try:
        os.mkdir(os.path.join(path_viejo,re.sub(".txt","",i)))
    except:
        pass
    with open(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/secciones",i),"r", encoding='utf8') as f:
        nombres=f.readlines()
    for j in nombres:
        #print(j)
        try:
            os.mkdir(re.sub("\n","",os.path.join(os.path.join(path_viejo,re.sub(".txt","",i)),j)))
        except Exception as e:
            print(e)
            #print(os.path.join(os.path.join(path_viejo,re.sub(".txt","",i)),j))
            pass

# Re-acomodo los excel en cada subcarpeta

In [ ]:
lista_coso=["salario_ambiente",
           "salario_control",
           "salario_deporte_turismo",
           "salario_hacienda",
           "salario_ilar",
           "salario_modernizacion",
           "salario_movilidad",
           "salario_obras",
           "salario_salud",
           "salario_spv"]

for carpeta in lista_coso:
    path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/"+carpeta
    lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

    path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/"+carpeta

    lista_dhea=natsorted(os.listdir(path_final))
    
    n=0
    numero_archivo=0
    for j in natsorted(os.listdir(path)):
        retiro_extension=re.sub(".xlsx","",j)
        numero=re.search("[0-999]",retiro_extension)
        inicio=numero.span()[0]
        coso=retiro_extension[inicio:]
        if int(coso) == lista[n]:
            shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        else:
            for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
                shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
            numero_archivo=numero_archivo+1
            
            #print("Aca deberia ir una funcion que mande todos los excel recogidos dentro de la carpeta\n")
            
            shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
            
            n=n+1
        n=n+1

    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))

# Manejo de excepciones

# Salario Cultura

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_cultura"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_cultura_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_cultura_perfecto_0.xlsx"))


path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_cultura"
lista=list(range(1,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=1
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))
except:
    pass

# Para el caso 2021_01

#shutil.move(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba/salario_cultura_perfecto_92.xlsx",os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura/40_TEATRO_LA_COMEDIA","salario_cultura_perfecto_92.xlsx"))

#try:
#    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
#    for rodri in lista_rodri:
#        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-2]))
#except:
#    pass



# Para el caso 2021_04

#shutil.move(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba/salario_cultura_perfecto_95.xlsx",os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_cultura/40_TEATRO_LA_COMEDIA","salario_cultura_perfecto_95.xlsx"))

#try:
#    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
#    for rodri in lista_rodri:
#        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-2]))
#except:
#    pass

# Salario Desarrollo Economico

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_desarrollo_economico"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_desarrollo_economico"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_desarrollo_economico_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_desarrollo_economico_perfecto_0.xlsx"))
shutil.move(os.path.join(path,"salario_desarrollo_economico_perfecto_1.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[1]),"salario_desarrollo_economico_perfecto_1.xlsx"))

path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_desarrollo_economico"
lista=list(range(2,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_desarrollo_economico"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=1
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))
except:
    pass

# Salario desarrollo humano

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_desarrollo_humano"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_desarrollo_humano"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_desarrollo_humano_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_desarrollo_humano_perfecto_0.xlsx"))
shutil.move(os.path.join(path,"salario_desarrollo_humano_perfecto_1.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[1]),"salario_desarrollo_humano_perfecto_1.xlsx"))



path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_desarrollo_humano"
lista=list(range(2,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_desarrollo_humano"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=2
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))
except:
    pass

# Salario Genero

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_genero_ddhh"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_genero_ddhh"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_genero_ddhh_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_genero_ddhh_perfecto_0.xlsx"))



path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_genero_ddhh"
lista=list(range(1,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_genero_ddhh"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=1
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))
except:
    pass

# Salario Gobierno

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_gobierno"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_gobierno"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_gobierno_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_gobierno_perfecto_0.xlsx"))
shutil.move(os.path.join(path,"salario_gobierno_perfecto_1.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[1]),"salario_gobierno_perfecto_1.xlsx"))



path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_gobierno"
lista=list(range(2,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_gobierno"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=2
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))
except:
    pass

# Salario intendencia

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_intendencia"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_intendencia"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_intendencia_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_intendencia_perfecto_0.xlsx"))
shutil.move(os.path.join(path,"salario_intendencia_perfecto_1.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[1]),"salario_intendencia_perfecto_1.xlsx"))



path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_intendencia"
lista=list(range(3,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",sorted(os.listdir(path),key=len)[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_intendencia"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=2
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    for rodri in lista_rodri:
        shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri),os.path.join(path_final,lista_dhea[-1]))
except:
    pass

# Salario Planeamiento

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_planeamiento"
lista=list(range(int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_planeamiento"

lista_dhea=natsorted(os.listdir(path_final))


shutil.move(os.path.join(path,"salario_planeamiento_perfecto_0.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[0]),"salario_planeamiento_perfecto_0.xlsx"))
shutil.move(os.path.join(path,"salario_planeamiento_perfecto_1.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[1]),"salario_planeamiento_perfecto_1.xlsx"))
shutil.move(os.path.join(path,"salario_planeamiento_perfecto_2.xlsx"),os.path.join(os.path.join(path_final,lista_dhea[2]),"salario_planeamiento_perfecto_2.xlsx"))


path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza/salario_planeamiento"
lista=list(range(4,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))
#print(lista)
# Para el caso 2020_09
#lista=list(range(3,int(re.sub(".xlsx","",natsorted(os.listdir(path))[-1])[re.search("[0-9]",re.sub(".xlsx","",natsorted(os.listdir(path))[-1])).span()[0]:])+1))

path_final=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos/salario_planeamiento"

lista_dhea=natsorted(os.listdir(path_final))

n=0
numero_archivo=3
for j in natsorted(os.listdir(path)):
    retiro_extension=re.sub(".xlsx","",j)
    numero=re.search("[0-999]",retiro_extension)
    inicio=numero.span()[0]
    coso=retiro_extension[inicio:]
    if int(coso) == lista[n]:
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
    else:
        for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")):
            shutil.move(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",k),os.path.join(path_final,lista_dhea[numero_archivo]))
        numero_archivo=numero_archivo+1
        shutil.move(os.path.join(path,j),os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",j))
        n=n+1
    n=n+1

try:
    lista_rodri=natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))
    #print(lista_rodri)
    for rodri in lista_rodri:
        path_inicio_pro=os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",rodri)
        path_final_pro=os.path.join(path_final,lista_dhea[-1])
        shutil.move(path_inicio_pro,path_final_pro)
except:
    pass

In [ ]:
shutil.rmtree(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/Excels_proceso_limpieza")

# PROPUESTA DE SOLUCION

In [ ]:
for k in natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos")):
    path=os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos",k)
    #print(path)
    for i in natsorted(os.listdir(path)):
        #print(i)
        if len(os.listdir(os.path.join(path,i))) == 1:
            path_inicio_1=os.path.join(os.path.join(path,i),natsorted(os.listdir(os.path.join(path,i)))[0])
            path_final_1=os.path.join(path,i)+".xlsx"
            shutil.move(path_inicio_1,path_final_1)
        else:
            path_1=os.path.join(os.path.join(path,i),natsorted(os.listdir(os.path.join(path,i)))[0])
            path_2=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"
            shutil.move(path_1,path_2)
            for j in natsorted(os.listdir(os.path.join(path,i))):
                shutil.move(os.path.join(os.path.join(path,i),j),path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba")
                path_df_1=os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))[0])
                path_df_2=os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))[1])
                df_1=pd.read_excel(path_df_1)
                df_2=pd.read_excel(path_df_2)
                df_final=pd.concat([df_1,df_2],ignore_index=True)
                os.remove(path_df_1)
                os.remove(path_df_2)
                df_final.to_excel(path_df_1,index=False,header=True)
            path_3=os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba",natsorted(os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/carpeta_prueba"))[0])
            path_4=os.path.join(path,i)+".xlsx"
            shutil.move(path_3,path_4)

In [ ]:
for i in os.listdir(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"):
    for j in os.listdir(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos",i)):
        try:
            os.rmdir(os.path.join(os.path.join(path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos",i),j))
        except:
            pass

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
for i in natsorted(os.listdir(path)):
    for j in natsorted(os.listdir(os.path.join(path,i))):
        df=pd.read_excel(os.path.join(os.path.join(path,i),j))
        final=df.shape[0]
        workbook= load_workbook(filename=os.path.join(os.path.join(path,i),j))
        sheet= workbook.active
        for k in range(1,final+2):
            celda=sheet.cell(row=k, column=1).value
            try:
                inicio_numero=re.search("[0-9]",celda).span()[0]
                nombre=celda[:inicio_numero]
                numero=celda[inicio_numero:]
                sheet["A"+str(k)] = nombre
                sheet["B"+str(k)] = numero
            except:
                pass
        workbook.save(filename=os.path.join(os.path.join(path,i),j))

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"
for i in natsorted(os.listdir(path)):
    #print(i)
    for j in natsorted(os.listdir(os.path.join(path,i))):
        df=pd.read_excel(os.path.join(os.path.join(path,i),j))
        final=df.shape[0]
        workbook= load_workbook(filename=os.path.join(os.path.join(path,i),j))
        sheet= workbook.active
        for k in range(1,final+2):
            celda_1=sheet.cell(row=k, column=2).value
            celda_2=sheet.cell(row=k, column=3).value
            celda_3=sheet.cell(row=k, column=4).value
            try:
                sheet["B"+str(k)]=re.sub(",",".",celda_1)
                sheet["C"+str(k)]=re.sub(",",".",celda_2)
                sheet["D"+str(k)]=re.sub(",",".",celda_3)
            except:
                pass
        workbook.save(filename=os.path.join(os.path.join(path,i),j))

In [ ]:
path=path_local+"/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/"+carpeta_especifica+"_Sueldos"

for i in natsorted(os.listdir(path)):
    for j in natsorted(os.listdir(os.path.join(path,i))):
        df=pd.read_excel(os.path.join(os.path.join(path,i),j))
        final=df.shape[0]
        workbook= load_workbook(filename=os.path.join(os.path.join(path,i),j))
        sheet= workbook.active
        for k in range(1,final+2):
            celda=sheet.cell(row=k, column=2).value
            if len(re.findall("\.",celda))>1:
                final=re.search("\.",celda).span()[1]
                sheet["B"+str(k)]=celda[:final+2]
            else:
                pass
        workbook.save(filename=os.path.join(os.path.join(path,i),j))

for i in natsorted(os.listdir(path)):
    for j in natsorted(os.listdir(os.path.join(path,i))):
        df=pd.read_excel(os.path.join(os.path.join(path,i),j))
        final=df.shape[0]
        workbook= load_workbook(filename=os.path.join(os.path.join(path,i),j))
        sheet= workbook.active
        for k in range(1,final+2):
            celda=sheet.cell(row=k, column=3).value
            if len(re.findall("\.",celda))>1:
                final=re.search("\.",celda).span()[1]
                sheet["C"+str(k)]=celda[:final+2]
            else:
                pass
        workbook.save(filename=os.path.join(os.path.join(path,i),j))
    
for i in natsorted(os.listdir(path)):
    for j in natsorted(os.listdir(os.path.join(path,i))):
        df=pd.read_excel(os.path.join(os.path.join(path,i),j))
        final=df.shape[0]
        workbook= load_workbook(filename=os.path.join(os.path.join(path,i),j))
        sheet= workbook.active
        for k in range(1,final+2):
            celda=sheet.cell(row=k, column=4).value
            if len(re.findall("\.",celda))>1:
                final=re.search("\.",celda).span()[1]
                sheet["D"+str(k)]=celda[:final+2]
            else:
                pass
        workbook.save(filename=os.path.join(os.path.join(path,i),j))

In [ ]:
Finalizacion_programa=datetime.now()
tiempo=Finalizacion_programa-Inicio_programa
from datetime import timedelta
tiempo_medido=str(timedelta(seconds=tiempo.seconds))[2:]
tiempo_medido

In [ ]:
#df_prueba=pd.read_excel("/home/rodrigo/Scrapper_Sueldos_Municipales/Gasto_Publico_Argentino_files/Salarios_Rosario/XLSX/2021_03_Sueldos/salario_ambiente/01_AREA_SERV_URBANOS_DIST_CENTRO.xlsx")

#df_prueba["costo_laboral"].sum()